# Exercise 7.4
Student: DC427182

In [1]:
# 01: importing the necessary dependencies
import pandas as pd
import geoplotlib
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox, DataAccessObject

In [2]:
# 02: reading the data
dataset = pd.read_csv('C:/Users/User/Documents/GitHub/gradest-1/DATAVIS/tutorials/Tutorial07/Datasets/flight_tracking.csv')

In [3]:
# 03: displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),latitude,longitude,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT


In [4]:
# 04: renaming columns latitude to lat and longitude to lon
dataset = dataset.rename(index=str, \
          columns={"latitude": "lat", "longitude": "lon"})

In [5]:
# 05: displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),lat,lon,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT


In [6]:
# 06: method to convert date and time to an unix timestamp
from datetime import datetime
def to_epoch(date, time):
    try:
        timestamp = round(datetime.strptime('{} {}'.\
                    format(date, time), \
                    '%Y/%m/%d %H:%M:%S.%f').timestamp())
        return timestamp
    except ValueError:
        return round(datetime.strptime('2017/09/11 17:02:06.418', \
                     '%Y/%m/%d %H:%M:%S.%f').timestamp())

In [7]:
# 07: create a new column called timestamp with the to_epoch method applied 
dataset['timestamp'] = dataset.apply(lambda x: to_epoch\
                                    (x['date'], x['time']), \
                                    axis=1)

In [8]:
# 08: displaying the first 5 rows of the dataset
dataset.head()

,hex_ident,altitude(feet),lat,lon,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign,timestamp
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN,1505120526
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN,1505120527
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT,1505120527
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT,1505120529
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT,1505120530


In [9]:
# 09: custom layer creation
class TrackLayer(BaseLayer):
    def __init__(self, dataset, bbox=BoundingBox.WORLD):
        self.data = dataset
        self.cmap = colorbrewer(self.data['hex_ident'], \
                                alpha=200)
        self.time = self.data['timestamp'].min()
        self.painter = BatchPainter()
        self.view = bbox
    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        df = self.data.where((self.data['timestamp'] \
                              > self.time) \
                              & (self.data['timestamp'] \
                              <= self.time + 180))
        for element in set(df['hex_ident']):
            grp = df.where(df['hex_ident'] == element)
            self.painter.set_color(self.cmap[element])
            x, y = proj.lonlat_to_screen(grp['lon'], grp['lat'])
            self.painter.points(x, y, 15, rounded=True)
        self.time += 1
        if self.time > self.data['timestamp'].max():
            self.time = self.data['timestamp'].min()
        self.painter.batch_draw()
        ui_manager.info('Current timestamp: {}'.\
                        format(epoch_to_str(self.time)))
       
    # bounding box that gets used when the layer is created
    def bbox(self):
        return self.view

In [10]:
# 10: bounding box for our view on Leeds
from geoplotlib.utils import BoundingBox
leeds_bbox = BoundingBox(north=53.8074, \
                         west=-3, \
                         south=53.7074 , \
                         east=0)

In [ ]:
# 11: displaying our custom layer using add_layer
data = DataAccessObject(dataset)
geoplotlib.add_layer(TrackLayer(data, bbox=leeds_bbox))
geoplotlib.show()